# Approach 1: Householder Transformation
Problem Transformation:
- Given the matrix $\mathbf{A}$ and vector $\mathbf{b}$, we'll apply Householder reflections to transform $\mathbf{A}$ into an upper triangular form:
$$
H_k H_{k-1} \cdots H_1 A=\left(\begin{array}{c}
R \\
0
\end{array}\right)
$$
where $\mathbf{H}_{-} \mathbf{i}$, for $\mathbf{1} \leq \mathbf{i} \leq \mathbf{k}$, are Householder reflectors.
- Similarly, transform the vector $\mathbf{b}$ :
$$
H_k H_{k-1} \cdots H_1 b=\left(\begin{array}{c}
c \\
d
\end{array}\right)
$$
where $\mathbf{c}$ is a vector in $\mathbb{R}^{\wedge} \mathbf{k}$ and $\mathbf{d}$ is a vector in $\mathbb{R}^{\wedge}(\mathbf{n}-\mathbf{k})$.
2. Solution:
- The resulting system of equations is now in the form $\mathbf{R x}=\mathbf{c}$.
- We'll solve for $\mathbf{x}$ using back-substitution.



In [54]:
def dot_product(a, b):
    """
    Compute the dot product of two vectors a and b.
    """
    if len(a) != len(b):
        raise ValueError("Vectors must have the same length")
    return sum(ai * bi for ai, bi in zip(a, b))

def vector_subtract(a, b):
    """
    Subtract vector b from vector a.
    """
    if len(a) != len(b):
        raise ValueError("Vectors must have the same length")
    return [ai - bi for ai, bi in zip(a, b)]

def vector_norm(v):
    """
    Compute the Euclidean norm (length) of a vector v.
    """
    return sum(vi ** 2 for vi in v) ** 0.5

def householder_reflect(v):
    """
    Compute the Householder reflection matrix for a given vector v.
    """
    norm_v = vector_norm(v)
    if norm_v == 0:
        raise ValueError("Input vector must be non-zero")
    v_hat = [vi / norm_v for vi in v]
    identity = [[1.0 if i == j else 0.0 for j in range(len(v))] for i in range(len(v))]
    outer_product = [[v_hat[i] * v_hat[j] for j in range(len(v))] for i in range(len(v))]
    H = [[identity[i][j] - 2 * outer_product[i][j] for j in range(len(v))] for i in range(len(v))]
    return H  # Return a list of lists

# Corrected solve_least_squares function
def solve_least_squares(A, b):
    """
    Solve the linear least squares problem min ||b - Ax||_2 using Householder transformations.
    """
    m, n = len(A), len(A[0])
    if m < n:
        raise ValueError("Overdetermined system required")
    
    # Apply Householder reflections to transform A into upper triangular form
    R = [row[:] for row in A]
    Q = [[1.0 if i == j else 0.0 for j in range(m)] for i in range(m)]
    for j in range(n):
        v = [0.0] * m
        for i in range(j, m):
            v[i] = R[i][j]
        v[j] += (1 if v[j] >= 0 else -1) * vector_norm(v)
        H = householder_reflect(v)
        R = [[sum(H[i][k] * R[k][j] for k in range(m)) for j in range(n)] for i in range(m)]
        Q = [[sum(H[i][k] * Q[k][j] for k in range(m)) for j in range(m)] for i in range(m)]
    
    # Solve the resulting upper triangular system of equations Rx = Q^Tb
    x = [0.0] * n
    Q_T_b = [sum(Q[i][k] * b[k] for k in range(m)) for i in range(m)]  # Compute Q^Tb
    for i in range(n - 1, -1, -1):
        x[i] = Q_T_b[i]
        for j in range(i + 1, n):
            x[i] -= R[i][j] * x[j]
        x[i] /= R[i][i]
    
    return x

# Given arrays A and b
A = [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]
b = [7.0, 8.0, 9.0]

# Solve the system
solution = solve_least_squares(A, b)
print("Solution x:", solution)


Solution x: [-6.000000000000002, 6.500000000000002]


#  Incremental Algorithm
Approach 2: Incremental Algorithm
Given the initial solution:
$$
x_{\min (n)}=\operatorname{argmin} x \in R k\left\|b_n-A_n x\right\|_2
$$
where $b_n$ and $A_n$ correspond to the first $n$ observations, we want to update the solution to:
$$
x_{\min }(n+1)=\operatorname{argmin} x \in R k\left\|b_{n+1}-A_{n+1} x\right\|_2
$$
as new observations arrive.

In [55]:
def dot_product(a, b):
    """
    Compute the dot product of two vectors a and b.
    """
    if len(a) != len(b):
        raise ValueError("Vectors must have the same length")
    return sum(ai * bi for ai, bi in zip(a, b))

def matrix_transpose(matrix):
    """
    Compute the transpose of a matrix.
    """
    return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]

def matrix_multiply(matrix1, matrix2):
    """
    Multiply two matrices.
    """
    if len(matrix1[0]) != len(matrix2):
        raise ValueError("Matrix dimensions do not match for multiplication")
    result = [[0.0] * len(matrix2[0]) for _ in range(len(matrix1))]
    for i in range(len(matrix1)):
        for j in range(len(matrix2[0])):
            result[i][j] = dot_product(matrix1[i], matrix_transpose(matrix2)[j])
    return result

def matrix_inverse(matrix):
    """
    Compute the inverse of a 2x2 matrix.
    """
    det = matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    if det == 0:
        raise ValueError("Matrix is not invertible")
    inv_det = 1.0 / det
    return [
        [matrix[1][1] * inv_det, -matrix[0][1] * inv_det],
        [-matrix[1][0] * inv_det, matrix[0][0] * inv_det]
    ]

def solve_least_squares_incremental(A, b):
    """
    Solve the linear least squares problem incrementally as new observations arrive.
    """
    n = len(A[0])  # Number of columns in A
    x = [0.0] * n  # Initialize solution with zeros
    
    for i in range(len(b)):
        # Update x incrementally using the new observation b_i
        A_i = [A[i]]  # New row of A
        b_i = [b[i]]  # New observation (as a single-element list)
        A_i_T = matrix_transpose(A_i)
        A_i_T_A_i = matrix_multiply(A_i_T, A_i)
        A_i_T_b_i = matrix_multiply(A_i_T, [b_i])  # Corrected this line
        x = [xi + A_i_T_b_i[j][0] / A_i_T_A_i[j][j] for j, xi in enumerate(x)]
    
    return x

# Given arrays A and b
A = [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]
b = [7.0, 8.0, 9.0]

# Solve the system incrementally
solution = solve_least_squares_incremental(A, b)
solution_rounded = [round(val, 2) for val in solution]
print("Solution x:", solution_rounded)


Solution x: [11.47, 7.0]


# # Regularized Linear Least Squares Problem
Given the underlying signal:
$$
x_{\text {true }}(t)=\sin (t)+t \cos ^2(t), \quad 0 \leq t \leq 4
$$
and evenly spaced time values $t_{-} 1=0$ to $t_{\_} s=4$, we want to reconstruct this signal using regularized least squares. We'll explore different values of the regularization parameter $\lambda$ and observe how the reconstruction quality varies with the number of samples $\mathrm{n}$.


Approach
1. Data Generation:
- Generate the noisy observations $\mathbf{b}$ from the true signal $\mathbf{x}_{-}\{\backslash$ text $\{$ true $\}\}(\mathbf{t})$.
- Add random noise to each observation.
2. Incremental Regularized Least Squares Solution:
- Initialize the solution $\mathbf{x}\{\{$ text\{RRS\}\} with zeros.
- As new observations arrive (for each $\mathbf{i}$ from 1 to $\mathbf{n}$ ):
- Update $\mathbf{x}\{\{$ text\{RRS\}\} incrementally using the new observation $\mathbf{b}[\mathbf{i}]$.
- Use different values of $\boldsymbol{\lambda}$ (e.g., 1, 10, 100, 1000).
3. Quality Assessment:
- Calculate the error (e.g., Euclidean norm) between $\mathbf{x}_{-}\left\{\right.$text\{RRS\}\} and $\mathbf{x}_{-}\{$text $\{$true $\}$.
- Observe how the error changes with $\mathbf{n}$ and the choice of $\boldsymbol{\lambda}$.

In [56]:
import math
import random

# Define the true signal x_true(t)
def true_signal(t):
    return math.sin(t) + t * math.cos(t)**2

# Generate noisy observations b from the true signal
n = 100  # Number of samples
t_values = [i * 4 / n for i in range(n)]

# Generate random noise
noise = []
for _ in range(n):
    u1, u2 = random.random(), random.random()
    z1 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)  # Box-Muller transform
    noise.append(0.1 * z1)  # Scale the noise by 0.1

b = [true_signal(t) + noise_val for t, noise_val in zip(t_values, noise)]

# Initialize the solution x_RRS with zeros
x_RRS = [0.0]  # Initialize with zeros

# Regularization parameter (you can adjust this)
lambda_val = 100

# Incremental update
for i in range(n):
    A_i = [[t_values[i]]]  # New observation
    b_i = b[i]
    
    # Update x_RRS incrementally using the new observation
    A_i_T = [[t_values[i]]]
    A_i_T_A_i = [[A_i_T[0][0]**2 + lambda_val]]
    A_i_T_b_i = [[A_i_T[0][0] * b_i]]
    
    x_RRS[0] += A_i_T_b_i[0][0] / A_i_T_A_i[0][0]

# Calculate the error between x_RRS and x_true
error = math.sqrt(sum((x_RRS[0] - true_signal(t))**2 for t in t_values))

print(f"Error between x_RRS and x_true: {error:.4f}")


Error between x_RRS and x_true: 22.5166
